---

In [1]:
import numpy as np
import pandas as pd
import sklearn

import plotly.express as px

---
---

# Modeling

---

In this part we will do some ML, implement different models to predict if a mail is a spam or not.

We will then process to a comparaison of the different models and conclude.